# AutoMM for Text - Quick Start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/autogluon/autogluon/blob/stable/docs/tutorials/multimodal/text_prediction/beginner_text.ipynb)
[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/autogluon/autogluon/blob/stable/docs/tutorials/multimodal/text_prediction/beginner_text.ipynb)



`MultiModalPredictor` can solve problems where the data are either image, text, numerical values, or categorical features.
To get started, we first demonstrate how to use it to solve problems that only contain text. We pick two classical NLP problems for the purpose of demonstration:

- [Sentiment Analysis](https://en.wikipedia.org/wiki/Sentiment_analysis)
- [Sentence Similarity](https://arxiv.org/abs/1910.03940)

Here, we format the NLP datasets as data tables where
the feature columns contain text fields and the label column contain numerical (regression) / categorical (classification) values.
Each row in the table corresponds to one training sample.

In [15]:
!pip uninstall autogluon autogluon.tabular autogluon.multimodal autogluon.core torch torchaudio -y


Found existing installation: autogluon.multimodal 1.1.1
Uninstalling autogluon.multimodal-1.1.1:
  Successfully uninstalled autogluon.multimodal-1.1.1
Found existing installation: autogluon.core 1.1.1
Uninstalling autogluon.core-1.1.1:
  Successfully uninstalled autogluon.core-1.1.1
Found existing installation: torch 2.4.1
Uninstalling torch-2.4.1:
  Successfully uninstalled torch-2.4.1
Found existing installation: torchaudio 2.4.1
Uninstalling torchaudio-2.4.1:
  Successfully uninstalled torchaudio-2.4.1


In [16]:
!pip install autogluon.multimodal


  Using cached autogluon.multimodal-1.1.1-py3-none-any.whl.metadata (12 kB)
  Using cached torch-2.3.1-cp310-cp310-win_amd64.whl.metadata (26 kB)
  Using cached autogluon.core-1.1.1-py3-none-any.whl.metadata (11 kB)
Using cached autogluon.multimodal-1.1.1-py3-none-any.whl (427 kB)
Using cached autogluon.core-1.1.1-py3-none-any.whl (234 kB)
Using cached torch-2.3.1-cp310-cp310-win_amd64.whl (159.8 MB)


In [17]:
%matplotlib inline

import numpy as np
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
np.random.seed(123)

## Sentiment Analysis Task

First, we consider the Stanford Sentiment Treebank ([SST](https://nlp.stanford.edu/sentiment/)) dataset, which consists of movie reviews and their associated sentiment.
Given a new movie review, the goal is to predict the sentiment reflected in the text (in this case a **binary classification**, where reviews are
labeled as 1 if they convey a positive opinion and labeled as 0 otherwise). Let's first load and look at the data,
noting the labels are stored in a column called **label**.

In [18]:
from autogluon.core.utils.loaders import load_pd
train_data = load_pd.load('https://autogluon-text.s3-accelerate.amazonaws.com/glue/sst/train.parquet')
test_data = load_pd.load('https://autogluon-text.s3-accelerate.amazonaws.com/glue/sst/dev.parquet')
subsample_size = 1000  # subsample data for faster demo, try setting this to larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head(10)

,sentence,label
43787,very pleasing at its best moments,1
16159,", american chai is enough to make you put away...",0
59015,too much like an infomercial for ram dass 's l...,0
5108,a stirring visual sequence,1
67052,cool visual backmasking,1
35938,hard ground,0
49879,"the striking , quietly vulnerable personality ...",1
51591,pan nalin 's exposition is beautiful and myste...,1
56780,wonderfully loopy,1
28518,"most beautiful , evocative",1


Above the data happen to be stored in the [Parquet](https://databricks.com/glossary/what-is-parquet) format, but you can also directly `load()` data from a [CSV](https://en.wikipedia.org/wiki/Comma-separated_values) file or other equivalent formats.
While here we load files from [AWS S3 cloud storage](https://docs.aws.amazon.com/AmazonS3/latest/dev/Welcome.html), these could instead be local files on your machine.
After loading, `train_data` is simply a [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html),
where each row represents a different training example.

### Training

To ensure this tutorial runs quickly, we simply call `fit()` with a subset of 1000 training examples and limit its runtime to approximately 1 minute.
To achieve reasonable performance in your applications, you are recommended to set much longer `time_limit` (eg. 1 hour), or do not specify `time_limit` at all (`time_limit=None`).

In [20]:
from autogluon.multimodal import MultiModalPredictor
import uuid
model_path = f"./tmp/{uuid.uuid4().hex}-automm_sst"
predictor = MultiModalPredictor(label='label', eval_metric='acc', path=model_path)
predictor.fit(train_data, time_limit=180)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          24
Pytorch Version:    2.3.1+cpu
CUDA Version:       CUDA is not available
Memory Avail:       18.14 GB / 31.91 GB (56.8%)
Disk Space Avail:   72.57 GB / 464.51 GB (15.6%)
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir c:\Users\Anthony\Desktop\Grad School\CMPE 255\Notebooks\tmp\640146

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 0, global step 3: 'val_acc' reached 0.56500 (best 0.56500), saving model to 'C:\\Users\\Anthony\\Desktop\\Grad School\\CMPE 255\\Notebooks\\tmp\\6401464a807440cfa3c2f25b4889b97f-automm_sst\\epoch=0-step=3.ckpt' as top 3
Epoch 0, global step 3: 'val_acc' reached 0.56500 (best 0.56500), saving model to 'C:\\Users\\Anthony\\Desktop\\Grad School\\CMPE 255\\Notebooks\\tmp\\6401464a807440cfa3c2f25b4889b97f-automm_sst\\epoch=0-step=3.ckpt' as top 3
Epoch 0, global step 3: 'val_acc' reached 0.56500 (best 0.56500), saving model to 'C:\\Users\\Anthony\\Desktop\\Grad School\\CMPE 255\\Notebooks\\tmp\\6401464a807440cfa3c2f25b4889b97f-automm_sst\\epoch=0-step=3.ckpt' as top 3
Epoch 0, global step 3: 'val_acc' reached 0.56500 (best 0.56500), saving model to 'C:\\Users\\Anthony\\Desktop\\Grad School\\CMPE 255\\Notebooks\\tmp\\6401464a807440cfa3c2f25b4889b97f-automm_sst\\epoch=0-step=3.ckpt' as top 3


...

Epoch 2, global step 16: 'val_acc' reached 0.83500 (best 0.83500), saving model to 'C:\\Users\\Anthony\\Desktop\\Grad School\\CMPE 255\\Notebooks\\tmp\\6401464a807440cfa3c2f25b4889b97f-automm_sst\\epoch=2-step=16.ckpt' as top 3
Epoch 2, global step 16: 'val_acc' reached 0.83500 (best 0.83500), saving model to 'C:\\Users\\Anthony\\Desktop\\Grad School\\CMPE 255\\Notebooks\\tmp\\6401464a807440cfa3c2f25b4889b97f-automm_sst\\epoch=2-step=16.ckpt' as top 3
Epoch 2, global step 16: 'val_acc' reached 0.83500 (best 0.83500), saving model to 'C:\\Users\\Anthony\\Desktop\\Grad School\\CMPE 255\\Notebooks\\tmp\\6401464a807440cfa3c2f25b4889b97f-automm_sst\\epoch=2-step=16.ckpt' as top 3
Epoch 2, global step 16: 'val_acc' reached 0.83500 (best 0.83500), saving model to 'C:\\Users\\Anthony\\Desktop\\Grad School\\CMPE 255\\Notebooks\\tmp\\6401464a807440cfa3c2f25b4889b97f-automm_sst\\epoch=2-step=16.ckpt' as top 3
Start to fuse 3 checkpoints via the greedy soup algorithm.


Predicting: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("c:\Users\Anthony\Desktop\Grad School\CMPE 255\Notebooks\tmp\6401464a807440cfa3c2f25b4889b97f-automm_sst")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).




Above we specify that: the column named **label** contains the label values to predict, AutoGluon should optimize its predictions for the accuracy evaluation metric,
trained models should be saved in the **automm_sst** folder, and training should run for around 60 seconds.

### Evaluation

After training, we can easily evaluate our predictor on separate test data formatted similarly to our training data.

In [ ]:
test_score = predictor.evaluate(test_data)
print(test_score)

By default, `evaluate()` will report the evaluation metric previously specified, which is `accuracy` in our example. You may also specify additional metrics, e.g. F1 score, when calling evaluate.

In [ ]:
test_score = predictor.evaluate(test_data, metrics=['acc', 'f1'])
print(test_score)

### Prediction

And you can easily obtain predictions from these models by calling `predictor.predict()`.

In [ ]:
sentence1 = "it's a charming and often affecting journey."
sentence2 = "It's slow, very, very, very slow."
predictions = predictor.predict({'sentence': [sentence1, sentence2]})
print('"Sentence":', sentence1, '"Predicted Sentiment":', predictions[0])
print('"Sentence":', sentence2, '"Predicted Sentiment":', predictions[1])

For classification tasks, you can ask for predicted class-probabilities instead of predicted classes.

In [ ]:
probs = predictor.predict_proba({'sentence': [sentence1, sentence2]})
print('"Sentence":', sentence1, '"Predicted Class-Probabilities":', probs[0])
print('"Sentence":', sentence2, '"Predicted Class-Probabilities":', probs[1])

We can just as easily produce predictions over an entire dataset.

In [ ]:
test_predictions = predictor.predict(test_data)
test_predictions.head()

### Save and Load

The trained predictor is automatically saved at the end of `fit()`, and you can easily reload it.

```{warning}

`MultiModalPredictor.load()` uses `pickle` module implicitly, which is known to be insecure. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling. Never load data that could have come from an untrusted source, or that could have been tampered with. **Only load data you trust.**

```

In [ ]:
loaded_predictor = MultiModalPredictor.load(model_path)
loaded_predictor.predict_proba({'sentence': [sentence1, sentence2]})

You can also save the predictor to any location by calling `.save()`.

In [ ]:
new_model_path = f"./tmp/{uuid.uuid4().hex}-automm_sst"
loaded_predictor.save(new_model_path)
loaded_predictor2 = MultiModalPredictor.load(new_model_path)
loaded_predictor2.predict_proba({'sentence': [sentence1, sentence2]})

### Extract Embeddings



You can also use a trained predictor to extract embeddings that maps each row of the data table to an embedding vector extracted from intermediate neural network representations of the row.

In [ ]:
embeddings = predictor.extract_embedding(test_data)
print(embeddings.shape)

Here, we use TSNE to visualize these extracted embeddings. We can see that there are two clusters corresponding to our two labels, since this network has been trained to discriminate between these labels.

In [ ]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2, random_state=123).fit_transform(embeddings)
for val, color in [(0, 'red'), (1, 'blue')]:
    idx = (test_data['label'].to_numpy() == val).nonzero()
    plt.scatter(X_embedded[idx, 0], X_embedded[idx, 1], c=color, label=f'label={val}')
plt.legend(loc='best')

## Sentence Similarity Task

Next, let's use MultiModalPredictor to train a model for evaluating how semantically similar two sentences are.
We use the [Semantic Textual Similarity Benchmark](https://paperswithcode.com/dataset/sts-benchmark) dataset for illustration.

In [ ]:
sts_train_data = load_pd.load('https://autogluon-text.s3-accelerate.amazonaws.com/glue/sts/train.parquet')[['sentence1', 'sentence2', 'score']]
sts_test_data = load_pd.load('https://autogluon-text.s3-accelerate.amazonaws.com/glue/sts/dev.parquet')[['sentence1', 'sentence2', 'score']]
sts_train_data.head(10)

In this data, the column named **score** contains numerical values (which we'd like to predict) that are human-annotated similarity scores for each given pair of sentences.

In [ ]:
print('Min score=', min(sts_train_data['score']), ', Max score=', max(sts_train_data['score']))

Let's train a regression model to predict these scores. Note that we only need to specify the label column and AutoGluon automatically determines the type of prediction problem and an appropriate loss function. Once again, you should increase the short `time_limit` below to obtain reasonable performance in your own applications.

In [ ]:
sts_model_path = f"./tmp/{uuid.uuid4().hex}-automm_sts"
predictor_sts = MultiModalPredictor(label='score', path=sts_model_path)
predictor_sts.fit(sts_train_data, time_limit=60)

We again evaluate our trained model's performance on separate test data. Below we choose to compute the following metrics: RMSE, Pearson Correlation, and Spearman Correlation.

In [ ]:
test_score = predictor_sts.evaluate(sts_test_data, metrics=['rmse', 'pearsonr', 'spearmanr'])
print('RMSE = {:.2f}'.format(test_score['rmse']))
print('PEARSONR = {:.4f}'.format(test_score['pearsonr']))
print('SPEARMANR = {:.4f}'.format(test_score['spearmanr']))

Let's use our model to predict the similarity score between a few sentences.

In [ ]:
sentences = ['The child is riding a horse.',
             'The young boy is riding a horse.',
             'The young man is riding a horse.',
             'The young man is riding a bicycle.']

score1 = predictor_sts.predict({'sentence1': [sentences[0]],
                                'sentence2': [sentences[1]]}, as_pandas=False)

score2 = predictor_sts.predict({'sentence1': [sentences[0]],
                                'sentence2': [sentences[2]]}, as_pandas=False)

score3 = predictor_sts.predict({'sentence1': [sentences[0]],
                                'sentence2': [sentences[3]]}, as_pandas=False)
print(score1, score2, score3)

Although the `MultiModalPredictor` currently supports classification and regression tasks, it can directly be used for
many NLP tasks if you properly format them into a data table. Note that there can be many text columns in this data table.
Refer to the [MultiModalPredictor documentation](../../../../api/autogluon.multimodal.MultiModalPredictor.fit.rst) to see all available methods/options.

Unlike `TabularPredictor` which trains/ensembles different types of models,
`MultiModalPredictor` focuses on selecting and finetuning deep learning based models.
Internally, it integrates with [timm](https://github.com/rwightman/pytorch-image-models) , [huggingface/transformers](https://github.com/huggingface/transformers),
[openai/clip](https://github.com/openai/CLIP) as the model zoo.

## Other Examples

You may go to [AutoMM Examples](https://github.com/autogluon/autogluon/tree/master/examples/automm) to explore other examples about AutoMM.

## Customization
To learn how to customize AutoMM, please refer to [Customize AutoMM](../advanced_topics/customization.ipynb).